# Traffic Flow Analysis - Interactive Notebook

Este notebook demonstra como usar o sistema de análise de tráfego para gerar insights sobre os padrões de mobilidade urbana do seu projeto de Hyperbolic Time Chamber.

## Análises Disponíveis:
- 🔥 **Heatmaps de tráfego** por horário e localização
- 📊 **Gráficos densidade vs. velocidade**
- 🚧 **Análise de gargalos** na rede viária
- 🗺️ **Padrões de mobilidade urbana** temporal
- ⚡ **Eficiência de rotas** calculadas
- 📈 **Indicadores de performance** da rede

## 1. Setup e Configuração

In [ ]:
# Importações necessárias
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Adicionar caminho do projeto
project_root = Path('.').absolute()
sys.path.append(str(project_root))

print(f"📂 Diretório de trabalho: {project_root}")

In [ ]:
# Importar módulos de análise
from config import CASSANDRA_CONFIG, VISUALIZATION_CONFIG
from data_sources.cassandra_source import CassandraDataSource
from data_sources.file_sources import CSVDataSource, JSONDataSource
from analysis.traffic_analyzer import TrafficAnalyzer
from visualization.traffic_viz import TrafficVisualizer
from reports.report_generator import TrafficReportGenerator

print("✅ Módulos importados com sucesso!")

## 2. Carregamento de Dados

Escolha a fonte de dados (descomente a opção desejada):

In [ ]:
# Opção 1: Dados do Cassandra (recomendado)
data_source = CassandraDataSource()
if data_source.connect():
    print("🔌 Conectado ao Cassandra com sucesso!")
    
    # Listar simulações disponíveis
    simulations = data_source.get_simulation_ids()
    print(f"📋 Simulações disponíveis: {simulations}")
    
    # Carregar dados da simulação mais recente (ou especificar uma)
    simulation_id = simulations[0] if simulations else None
    print(f"🎯 Usando simulação: {simulation_id}")
    
    # Carregar dados de fluxo de veículos
    data = data_source.get_vehicle_flow_data(
        simulation_id=simulation_id,
        limit=5000  # Limitar para análise interativa
    )
else:
    print("❌ Falha na conexão com Cassandra")
    data = pd.DataFrame()

# # Opção 2: Dados de arquivo CSV
# csv_source = CSVDataSource()
# data = csv_source.load_vehicle_flow_data(combine_files=True)

# # Opção 3: Dados de arquivo JSON
# json_source = JSONDataSource()
# data = json_source.load_vehicle_flow_data(combine_files=True)

print(f"📊 Carregados {len(data):,} registros de fluxo de veículos")
if not data.empty:
    print(f"📅 Período: {data['timestamp'].min()} até {data['timestamp'].max()}")
    print(f"🚗 Veículos únicos: {data['car_id'].nunique() if 'car_id' in data.columns else 'N/A'}")
    print(f"🛣️ Links únicos: {data['link_id'].nunique() if 'link_id' in data.columns else 'N/A'}")

## 3. Análise Exploratória dos Dados

In [ ]:
# Visualizar primeiras linhas
print("📋 Estrutura dos dados:")
display(data.head())

print("\n📊 Informações dos dados:")
display(data.info())

print("\n📈 Estatísticas descritivas:")
display(data.describe())

In [ ]:
# Verificar tipos de eventos
if 'event_type' in data.columns:
    print("📋 Distribuição de tipos de eventos:")
    event_counts = data['event_type'].value_counts()
    display(event_counts)
    
    # Gráfico simples
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    event_counts.plot(kind='bar')
    plt.title('Distribuição de Tipos de Eventos')
    plt.xlabel('Tipo de Evento')
    plt.ylabel('Quantidade')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 4. Análise de Tráfego Completa

In [ ]:
# Inicializar analisador
analyzer = TrafficAnalyzer(data)

# Métricas básicas
print("📊 Calculando métricas básicas...")
basic_metrics = analyzer.calculate_basic_metrics()
display(basic_metrics)

In [ ]:
# Padrões de tráfego
print("🕐 Analisando padrões temporais...")
traffic_patterns = analyzer.analyze_traffic_patterns()
display(traffic_patterns)

In [ ]:
# Identificar gargalos
print("🚧 Identificando gargalos...")
bottlenecks = analyzer.identify_bottlenecks()

print(f"Encontrados {len(bottlenecks)} gargalos")
if bottlenecks:
    print("\nTop 5 gargalos mais críticos:")
    for i, bottleneck in enumerate(bottlenecks[:5], 1):
        print(f"{i}. Link {bottleneck['link_id']}: "
              f"Severidade {bottleneck['severity_score']:.3f}, "
              f"Velocidade média {bottleneck['avg_speed']:.1f} km/h")

In [ ]:
# Análise de eficiência de rotas
print("⚡ Analisando eficiência de rotas...")
route_efficiency = analyzer.analyze_route_efficiency()
display(route_efficiency)

In [ ]:
# Indicadores de mobilidade
print("🗺️ Calculando indicadores de mobilidade...")
mobility_indicators = analyzer.calculate_mobility_indicators()
display(mobility_indicators)

## 5. Visualizações Interativas

In [ ]:
# Inicializar visualizador
visualizer = TrafficVisualizer(data)

# Heatmap de tráfego
print("🔥 Gerando heatmap de tráfego...")
heatmap_fig = visualizer.create_traffic_heatmap()
heatmap_fig.show()

In [ ]:
# Gráfico velocidade vs densidade
print("📊 Criando gráfico velocidade vs densidade...")
speed_density_fig = visualizer.create_speed_density_plot()
speed_density_fig.show()

In [ ]:
# Análise de gargalos
if bottlenecks:
    print("🚧 Visualizando análise de gargalos...")
    bottleneck_fig = visualizer.create_bottleneck_analysis(bottlenecks)
    bottleneck_fig.show()

In [ ]:
# Padrões de mobilidade
print("🗺️ Visualizando padrões de mobilidade...")
mobility_fig = visualizer.create_mobility_patterns()
mobility_fig.show()

In [ ]:
# Dashboard de eficiência de rotas
if route_efficiency:
    print("⚡ Criando dashboard de eficiência...")
    efficiency_fig = visualizer.create_route_efficiency_dashboard(route_efficiency)
    efficiency_fig.show()

## 6. Relatório Executivo

In [ ]:
# Compilar todos os resultados
analysis_results = {
    'basic_metrics': basic_metrics,
    'traffic_patterns': traffic_patterns,
    'bottlenecks': bottlenecks,
    'route_efficiency': route_efficiency,
    'mobility_indicators': mobility_indicators
}

# Gerar relatório executivo
reporter = TrafficReportGenerator()
executive_summary = reporter.generate_executive_summary(analysis_results)

print("📋 RESUMO EXECUTIVO")
print("=" * 50)

overview = executive_summary['overview']
print(f"🚗 Total de veículos analisados: {overview['total_vehicles_analyzed']:,}")
print(f"📊 Total de eventos capturados: {overview['total_events_captured']:,}")
print(f"🛣️ Links monitorados: {overview['unique_links_monitored']:,}")

findings = executive_summary['key_findings']
print(f"\n🔍 DESCOBERTAS PRINCIPAIS:")
print(f"• Velocidade média da rede: {findings['average_speed']:.1f} km/h")
print(f"• Gargalos identificados: {findings['bottleneck_count']}")
print(f"• Eficiência da rede: {findings['network_efficiency']:.1%}")

if findings['peak_traffic_hours']:
    print(f"• Horários de pico: {', '.join(map(str, findings['peak_traffic_hours']))}")

print(f"\n💡 RECOMENDAÇÕES:")
for i, rec in enumerate(executive_summary['recommendations'], 1):
    print(f"{i}. {rec}")

if executive_summary['critical_issues']:
    print(f"\n⚠️ QUESTÕES CRÍTICAS:")
    for issue in executive_summary['critical_issues']:
        print(f"• {issue['title']}: {issue['description']}")

## 7. Salvar Resultados

In [ ]:
# Criar diretório de saída
output_dir = Path('output') / f"analysis_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Salvando resultados em: {output_dir}")

# Salvar visualizações
viz_paths = visualizer.save_all_visualizations(
    analysis_results,
    output_dir=output_dir / "visualizations"
)

print(f"📈 Visualizações salvas:")
for name, path in viz_paths.items():
    print(f"  • {name}: {path}")

# Salvar relatórios
report_paths = reporter.generate_all_reports(
    analysis_results,
    viz_paths
)

print(f"\n📋 Relatórios gerados:")
for format_type, path in report_paths.items():
    print(f"  • {format_type.upper()}: {path}")

print(f"\n✅ Análise completa finalizada!")
if 'html' in report_paths:
    print(f"\n🌐 Abra o relatório HTML no navegador:")
    print(f"   file://{Path(report_paths['html']).absolute()}")

## 8. Análises Personalizadas

Use as células abaixo para análises específicas:

In [ ]:
# Análise personalizada 1: Links mais utilizados
if 'link_id' in data.columns:
    print("🛣️ TOP 10 LINKS MAIS UTILIZADOS:")
    top_links = data['link_id'].value_counts().head(10)
    
    for i, (link_id, count) in enumerate(top_links.items(), 1):
        avg_speed = data[data['link_id'] == link_id]['calculated_speed'].mean() if 'calculated_speed' in data.columns else 0
        print(f"{i:2d}. Link {link_id}: {count:,} veículos (vel. média: {avg_speed:.1f} km/h)")

In [ ]:
# Análise personalizada 2: Distribuição temporal detalhada
if 'timestamp' in data.columns:
    print("🕐 ANÁLISE TEMPORAL DETALHADA:")
    
    data['hour'] = data['timestamp'].dt.hour
    hourly_stats = data.groupby('hour').agg({
        'car_id': 'count',
        'calculated_speed': 'mean'
    }).round(2)
    
    hourly_stats.columns = ['Veículos', 'Vel_Média']
    display(hourly_stats)

In [ ]:
# Limpeza final
if 'data_source' in locals() and hasattr(data_source, 'close'):
    data_source.close()
    print("🔌 Conexão com Cassandra fechada")

print("\n🎉 Notebook concluído com sucesso!")
print("\n📚 Para mais análises:")
print("   • Modifique os parâmetros das funções")
print("   • Experimente diferentes filtros nos dados")
print("   • Combine múltiplas simulações")
print("   • Explore os módulos de visualização")

---

## 🚀 Próximos Passos

1. **Análise Comparativa**: Compare múltiplas simulações
2. **Análise Temporal**: Analise evolução ao longo do tempo
3. **Otimização**: Use insights para ajustar parâmetros de simulação
4. **Machine Learning**: Implemente modelos preditivos
5. **Integração**: Conecte com outros sistemas de análise

Para suporte técnico, consulte a documentação dos módulos ou entre em contato com a equipe de desenvolvimento.